In [ ]:
import os,sys
import hashlib
import json
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(parent_dir)
from postgres import Postgres

pg = Postgres()

def create_table_if_not_exists():
    pg.insert("""
        CREATE TABLE IF NOT EXISTS text_chunks (
        chunk_id VARCHAR NOT NULL,  -- Must be VARCHAR to match the referenced type
        document_id TEXT NOT NULL,
        document_geo_boundary GEOMETRY,
        geo_scope TEXT,
        chunk_sections TEXT,
        chunk_text TEXT,
        chunk_geom GEOMETRY,
        distance_from_document_geom DOUBLE PRECISION,
        openai_topic_labels JSONB,
        openai_geo_labels JSONB,
        experiment TEXT,
        lpa TEXT,
        council_type TEXT,
        neighbourhood TEXT,
        document_title TEXT,
        document_category TEXT,
        document_start_year INT,
        document_end_year INT,
        chunkking_details JSONB,
        chunk_vector VECTOR(1536),
        
        -- Define `chunk_id` as a foreign key referencing `langchain_pg_embedding(id)`
        CONSTRAINT fk_chunk_id
            FOREIGN KEY (chunk_id) REFERENCES langchain_pg_embedding(id)
    );
    """)


def add_data():
    # Step 2: Query the data from `langchain_pg_embedding`
    print('cycle')
    fetch_unprocessed_rows_query = """
        SELECT * 
        FROM langchain_pg_embedding AS l
        WHERE NOT EXISTS (
            SELECT 1 
            FROM text_chunks AS t
            WHERE t.chunk_id = l.id
        )
        AND cmetadata->>'chunker' = 'sherpa'
        LIMIT 20;
    """ 
    rows = pg.query(fetch_unprocessed_rows_query)

    # Step 3: Process each row and insert into `text_chunks`
    for row in rows:
        row = dict(row)
        # Access fields directly by column name due to RealDictCursor
        cmetadata = row['cmetadata']

        # Build the transformed data with Python, handling missing fields with `.get()`
        document_id = hashlib.md5(
            (cmetadata.get('document_title', '') +
            str(cmetadata.get('start_year', '')) +
            str(cmetadata.get('end_year', '')) +
            cmetadata.get('lpa', '')).encode('utf-8')
        ).hexdigest()
        
        data = {
            "chunk_id": row['id'],
            "document_id": document_id,
            "document_geo_boundary": None,
            "geo_scope": "",
            "chunk_sections": cmetadata.get('sections', ''),
            "chunk_text": cmetadata.get('text', ''),
            "distance_from_document_geom": None,
            "openai_topic_labels": json.dumps(cmetadata.get('openai_labels', {})),  
            "openai_geo_labels": json.dumps(cmetadata.get('openai_geo_labels', {})),
            "experiment": cmetadata.get('experiment', ''),
            "lpa": cmetadata.get('lpa', ''),
            "council_type": cmetadata.get('council type', ''),
            "neighbourhood": cmetadata.get('neighbourhood', ''),
            "document_title": cmetadata.get('document_title', ''),
            "document_category": cmetadata.get('category', ''),
            "document_start_year": int(cmetadata.get('start_year', 0)),
            "document_end_year": int(cmetadata.get('end_year', 0)),
            "chunkking_details": json.dumps({
                "chunker": cmetadata.get('chunker', ''),
                "file": cmetadata.get('file', ''),
                "url": cmetadata.get('url', ''),
                "page": int(cmetadata.get('page', 0)),
                "block_idx": int(cmetadata.get('block_idx', 0)),
                "level": int(cmetadata.get('level', 0)),
                "coalesced_with": cmetadata.get('coalesced_with', ''),
                "notes": cmetadata.get('notes', '')
            }),
            "chunk_vector": row['embedding']  # Adjusted to use column name 'vector'
        }

        # Insert into `text_chunks`
        insert_query = """
            INSERT INTO text_chunks (
                chunk_id,
                document_id, document_geo_boundary, geo_scope, chunk_sections, chunk_text,
                chunk_geom, distance_from_document_geom, openai_topic_labels, openai_geo_labels,
                experiment, lpa, council_type, neighbourhood, document_title, document_category,
                document_start_year, document_end_year, chunkking_details, chunk_vector
            ) VALUES (
                %(chunk_id)s,
                %(document_id)s, %(document_geo_boundary)s, %(geo_scope)s, %(chunk_sections)s,
                %(chunk_text)s, %(chunk_geom)s, %(distance_from_document_geom)s,
                %(openai_topic_labels)s, %(openai_geo_labels)s, %(experiment)s, %(lpa)s,
                %(council_type)s, %(neighbourhood)s, %(document_title)s, %(document_category)s,
                %(document_start_year)s, %(document_end_year)s, %(chunkking_details)s, %(chunk_vector)s
            );
        """
        pg.insert(insert_query, data)


for i in range(100):
    add_data()

cycle
